<a href="https://colab.research.google.com/github/brendenwest/ad450/blob/master/9_tree_based_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Tree-based Models

### Reading
- Geron - chapter 6 ([Decision trees](https://learning.oreilly.com/library/view/hands-on-machine-learning/9781491962282/ch06.html#trees_chapter)) & chapter 7 ([Ensembles](https://learning.oreilly.com/library/view/hands-on-machine-learning/9781491962282/ch07.html#ensembles_chapter))

### Tutorials
- https://www.datacamp.com/community/tutorials/decision-tree-classification-python
- https://www.datacamp.com/community/tutorials/kaggle-tutorial-machine-learning (Decision Tree Classifiers)
- https://www.datacamp.com/community/tutorials/random-forests-classifier-python


### Learning Outcomes
- Decision tree models
- Measuring entropy & information gain
- Using decision trees for regression
- Ensemble learning
- Random forests


## Overview

Decision trees are flowchart-like tree structures, where each node represents a feature (attribute). Branches represent a decision rule and each leaf node represents the decision outcome.


**Pros:**
- training time is faster than more complex ML algorithms. Time complexity depends on number of records and number of attributes
- Easy to interpret & visualize
- can easily capture non-linear patterns
- can handle high-dimensional data with good accuracy
- can be used for feature engineering 
- does not depend on probability distribution assumptions

**Cons:**
- sensitive to noisy data & prone to overfit
- sensitive to variation in dataset (can be addressed with **bagging** & **boosting**)
- biased with unbalanced dataset

How the Decision Tree algorithm works:

1.   Selecting the best attribute using an Attribute Selection Measure (ASM)
2.   Make that attribute a decision node & partition the dataset on it
3.   Repeast this process recursively for each child node until one of these conditions match:
     - all tuples belong to the same attribute value
     - there are no more attributes
     - there are no more instances


## Attribute Selection Measures (ASM)

Refers to methods for partitioning a dataset in the way that best explains it. 

Each attribute is ranked by how well it explains the dataset, and the attribute with the best score is selected.

For attributes with continuous values, it's necessary to define split points.


### Information Gain

Information gain is a decrease in **entropy** (randomness or impurity) in a dataset.

The Information Gain ASM computes the difference between entropy before split and average entropy after split of the dataset based on given attribute values.

The attribute with highest information gain - at a given decision node - is chosen as the splitting attribute.

Information Gain is biased for the attribute with a large number of distinct values.

### Gain Ratio

Gain ratio handles the issue of bias by normalizing the information gain using Split Info.

### Gini Index


## Optimizing Decision Tree Performance

## Visualizing Decision Trees